In [1]:
import pandas
import datetime

In [2]:
input_path = "./output data/"
input_violent_crime_by_city_and_state_with_geographical_location = "output_focus_group_and_incident"

In [3]:
analysis_dataframe =pandas.DataFrame()
analysis_dataframe = pandas.read_csv(input_path+input_violent_crime_by_city_and_state_with_geographical_location+".csv")

In [4]:
analysis_dataframe.head()

,bias,center_lat,center_long,city,city_population,northeast_lat,northeast_long,southwest_lat,southwest_long,state,year
0,Anti-Male Homosexual,33.425510,-111.940005,TEMPE,156788.0,33.463984,-111.877226,33.319861,-111.978976,AZ,1996.0
1,Anti-Male Homosexual,34.483901,-114.322455,LAKE HAVASU CITY,34657.0,34.589585,-114.246145,34.430294,-114.375789,AZ,1996.0
2,Anti-Female Homosexual,32.222607,-110.974711,TUCSON,472385.0,32.320166,-110.708204,31.991654,-111.059406,AZ,1996.0
3,Anti-Male Homosexual,33.448377,-112.074037,PHOENIX,0.0,33.920570,-111.926046,33.290260,-112.324056,AZ,1996.0
4,Anti-Male Homosexual,37.871523,-122.273042,BERKELEY,101250.0,37.906689,-122.234196,37.835637,-122.368679,CA,1996.0


In [5]:
analysis_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43017 entries, 0 to 43016
Data columns (total 11 columns):
bias               43017 non-null object
center_lat         43017 non-null float64
center_long        43017 non-null float64
city               43017 non-null object
city_population    43017 non-null float64
northeast_lat      43017 non-null float64
northeast_long     43017 non-null float64
southwest_lat      43017 non-null float64
southwest_long     43017 non-null float64
state              43017 non-null object
year               43017 non-null float64
dtypes: float64(8), object(3)
memory usage: 3.6+ MB


## population growth vs hate crimes over time

#### change in population of cities vs hate crime rate

### create summary dataframe : crimes by location population by year

analysis_dataframe['year_and_city']=0

for current_row in range(0, analysis_dataframe.shape[0]-1, 1 ) :
    if (current_row>0) & (current_row%100 == 0 ):
        print( current_row )
    analysis_dataframe.iloc[current_row,10] = str(analysis_dataframe.iloc[current_row,7]) +"-"+ analysis_dataframe.iloc[current_row,4]


analysis_dataframe.head(10)

#### Load custom function library includes many functions to facilitate linear regression analysis and calculation of related metrics.

In [6]:
def clean_numerics( target_string ):
    #convert a number possibly containing characters that Python can't inherently convert ($,% etc)
    #   into a float if it contains a decimal point or 'e' (in the correct place), otherwise an integer
    
    if type( target_string ) != str:
        raise TypeError
    
    #string_length = range(0,len(target_string)) #I think this line is obsolete and should be removed
    working_copy = target_string
    for current_character in target_string :
            if current_character in ( '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '.', '-', '+', 'e' ) :
                if (current_character=='.') | (current_character=='e') :
                    return_type = 'float'
            else :
                working_copy = working_copy.replace( current_character, '' )            
    
    if ('.' in working_copy) | ('e' in working_copy) :
        try :
            return float( working_copy )
        except :
            raise ValueError
    
    try :
        return int( working_copy )
    except :
        raise ValueError
     
    #not sure how we'd get here, but if we do, don't wipe out the original value if the var is saved back to itself.
    return target_string


def convert_percent_with_symbol_to_int( percent_string_value):
    if type( percent_string_value ) != str:
        raise TypeError
        
    return int(percent_string_value.replace( '%', '' ))


def convert_percent_series_from_string_to_int( dataframe, percent_string_value ):
    if type( Type( dataframe) ) != pandas.core.frame.DataFrame:
        raise TypeError
        
    if type( percent_string_value ) != str:
        raise TypeError
    
    dataframe[ target_fieldname ] = dataframe[ target_fieldname ].map( convert_percent_with_symbol_to_int )
    dataframe[ target_fieldname ].astype(int)
    
    return dataframe


def show_characteristics( dataframe ) :
    if type( dataframe) != pandas.core.frame.DataFrame :
        raise TypeError    
    
    print( "Data shape : ", dataframe.shape )
    print ( "\n\n")
    
    print ( "Descibe Data : \n", dataframe.describe() )
    print ( "\n\n")
    
    print( "Show info about Data : \n", dataframe.info() )
    print ( "\n\n" )

    print( "Summarize Data : \n", dataframe.head(3).T )
    print ( "\n\n" )
    
    print( "Features with NaNs & Count\n")
    for current_series in dataframe :
        if dataframe[current_series].isnull().sum() > 0 :
            print( current_series, " ", dataframe[current_series].isnull().sum() )
    print ( "\n\n" )
    
    
def find_errors_in_numeric_fields( dataframe, target_fieldname ) :
    if type( dataframe ) != pandas.core.frame.DataFrame:
        raise TypeError    
    
    if type( target_fieldname ) == str :
        raise TypeError
    
    erroneous_values = list()
    for current_value in dataframe[ target_fieldname ] :   
        try :
            float( current_value )
        except :
            erroneous_values.append ( current_value )
    return erroneous_values[:4]


def show_data_outside_boundaries( dataframe, target_fieldname , low, high ):
    if type( dataframe ) != pandas.core.frame.DataFrame :
        raise TypeError    
    
    if type( target_fieldname ) != str :
        raise TypeError
    
    if type( low ) != float and  type( low ) != int :
        raise TypeError
    
    if type( high ) != float and  type( high ) != int :
        raise TypeError  
    
    return dataframe[ (dataframe[ target_fieldname ] < low) & (dataframe[ target_fieldname ] > high ) ]

        
def get_linear_regression_model( independent_variables_dataframe ) :
    if type( independent_variables_dataframe ) != pandas.core.frame.DataFrame :
        raise TypeError    

    linear_regression_model.predict(independent_variables_dataframe )
    
    return linear_regression_model
 
    
def get_predictions_and_residuals( dataframe, linear_regression_model, list_of_features, dependent_variable_name )  : 
    if type( dataframe ) != pandas.core.frame.DataFrame :
        raise TypeError    

    if type( linear_regression_model ) != LinearRegression :
        raise TypeError 
    
    if type( list_of_features  ) !=  list :
        raise TypeError
        
    if type( dependent_variable_name ) !=  str :
        raise TypeError    
    
    independent_variables_dataframe = dataframe[ list_of_features ]
    dependent_variable_series = dataframe[ dependent_variable_name ]
    predictions = linear_regression_model.predict( independent_variables_dataframe )
    residuals = dependent_variable_series  - predictions
    
    return predictions, residuals
    
    
def plot_scatter_plot_of_predicitons_vs_residuals( predictions, residuals ) :
    if type( predictions ) !=  numpy.ndarray :
        raise TypeError 

    if type( residuals ) !=  pandas.core.series.Series :
        raise TypeError     
    
    print( "Scatter plot of predicitons vs residuals ")
    
    matplotlib.pyplot.scatter(predictions, residuals)
    matplotlib.pyplot.axhline( 0, color='red')
    matplotlib.pyplot.show()
    
    
def plot_coeficient_of_linear_regression_model( linear_regression_model ) :
    if type( linear_regression_model ) !=  sklearn.linear_model.base.LinearRegression :
        raise TypeError 

    print( "coeficient of linear regression model ")
    linear_regression_model.coef_
    matplotlib.pyplot.show()
    
    
def plot_intercept_of_regression_model( linear_regression_model ) :
    if type( linear_regression_model ) !=  sklearn.linear_model.base.LinearRegression :
        raise TypeError

    linear_regression_model.intercept_   #NOTE : intercept_  aka BIAS
    matplotlib.pyplot.show()

    
def plot_histogram_of_epsilon( dependent_variable_series, predictions ) :
    if type( dependent_variable_name ) !=  str :
        raise TypeError    

    if type( predictions ) !=  numpy.ndarray :
        raise TypeError 
    
    residuals = dependent_variable_series  - predictions
    error_due_to_the_inherent_noisiness_of_the_universe = residuals.mean()  #this is epsilon, 
                                                                            #we want it as small as possible
    print( "epsilon = ", error_due_to_the_inherent_noisiness_of_the_universe )  
    residuals.hist()  #recall epsilon should be symmetrical; if histogram is symmetrical outliers are rare.
    matplotlib.pyplot.show()

def plot_scatter_of_hot_encoded_series( dataframe, series_prefix, xlabel="", ylabel="", title="" ) :
    
    
    
    if type( series_prefix ) !=  str :
        raise TypeError    
    
    if type( xlabel ) !=  str :
        raise TypeError 
        
    if type( ylabel ) !=  str :
        raise TypeError 
    
    if type( title ) !=  str :
        raise TypeError
        
    binary_fields = pandas.DataFrame({})
    
    for current_series in dataframe :
        if series_prefix in current_series :
            binary_fields[ current_series ] = [dataframe[current_series].sum(), 
                                               dataframe[ 
                                                   (dataframe[current_series]==1) & 
                                                   (dataframe[current_series]>0)][ 'SalePrice'].mean(),
                                               dataframe[ 
                                                   (dataframe[current_series]==1) & 
                                                   (dataframe[current_series]>0)][ 'SalePrice'].max(),
                                               dataframe[ 
                                                   (dataframe[current_series]==1) & 
                                                   (dataframe[current_series]>0)][ 'SalePrice'].min()
                                              ]
        
    features = binary_fields.T
    names = binary_fields.columns
    count = binary_fields.get_values()[0]
    average_cost = binary_fields.get_values()[1]/1000
    exp_value = binary_fields.get_values()[2]
    matplotlib.pyplot.scatter( count, names, alpha=0.2, s=average_cost, c="r" );
    matplotlib.pyplot.xlabel(xlabel)
    matplotlib.pyplot.ylabel(ylabel)
    matplotlib.pyplot.title( title )
    matplotlib.pyplot.colorbar()
    
    
def plot_histogram_of_dependent_variable( dataframe, dependent_variable_series ) :
    if type( dataframe ) != pandas.core.frame.DataFrame :
        raise TypeError
          
    print( "type( dependent_variable_series ) = ", type( dependent_variable_series ))

    dependent_variable_series.hist() 
    matplotlib.pyplot.show()  

def subplot_dataset(dataframe, list_of_columns, list_of_titles, list_of_xlabels, list_of_ylabels ) :
    if type( dataframe ) != pandas.core.frame.DataFrame :
        raise TypeError
        
        
        
        
    if type( list_of_titles ) !=  list :
        raise TypeError    
    if type( list_of_xlabels ) !=  list :
        raise TypeError    
    if type( list_of_ylabels ) !=  list :
        raise TypeError    
    
    nrows = int(numpy.ceil(len(list_of_columns)/2)) # Makes sure you have enough rows
    fig,ax = matplotlib.pyplot.subplots(nrows=10, ncols=2) # You'll want to specify your figsize
    ax = ax.ravel() # Ravel turns a matrix into a vector, which is easier to iterate
    matplotlib.pyplot.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)
    for i, column in enumerate(list_of_columns): # Gives us an index value to get into all our lists
        seaborn.countplot(dataframe[column], ax=ax[0])
        fig.show()
        
def subplot_histograms(dataframe, list_of_columns, list_of_titles, list_of_xlabels, list_of_ylabels ):
    if type( dataframe ) != pandas.core.frame.DataFrame :
        raise TypeError
          
    print( "type( list_of_titles ) = ", type( list_of_titles ))
    print( "type( list_of_xlabels ) = ", type( list_of_xlabels ))
    print( "type( list_of_ylabels ) = ", type( list_of_ylabels ))

    
    nrows = int(numpy.ceil(len(list_of_columns)/2)) # Makes sure you have enough rows
    fig,ax = matplotlib.pyplot.subplots(nrows=nrows, ncols=2) # You'll want to specify your figsize
    ax = ax.ravel() # Ravel turns a matrix into a vector, which is easier to iterate
    matplotlib.pyplot.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)
    for i, column in enumerate(list_of_columns): # Gives us an index value to get into all our lists
        #ax[i].hist(dataframe[column]) # feel free to add more settings
        ax[i].plot(dataframe[column]) # feel free to add more settings
        # Set titles, labels, etc here for each subplot
        ax[i].set_title(list_of_titles[i])     # Set titles, labels, etc here for each subplot         
        ax[i].set_xlabel(list_of_xlabels[i])
        ax[i].set_ylabel(list_of_ylabels[i])
        ax[i].set_ylim((1,25))


def calculate_mean_absolute_error( residuals ) :
    if type( residuals ) !=  pandas.core.series.Series :
        raise TypeError 

    mean_absolute_error = numpy.abs(residuals).mean()
    print( "Mean Absolute Error (MAE) = ", mean_absolute_error, "   (lower is better)" ) #recall : residuals = error
    return mean_absolute_error


def calculate_residual_sum_of_squares( residuals ) :
    if type( residuals ) !=  pandas.core.series.Series :
        raise TypeError 

    residuals_squared = residuals**2
    residual_sum_of_squares = residuals_squared.sum()
    print( "Residual Sum of Squares ( RSS ) = ", residual_sum_of_squares, "   (lower is better)" )
    return residual_sum_of_squares


def calculate_mean_squared_error( residual_sum_of_squares, predictions ) :
    if type( residual_sum_of_squares ) !=  numpy.float64 :
        raise TypeError
    
    if type( predictions ) !=  numpy.ndarray :
        print( "predictions should be of type numpy.ndarray" )
        raise TypeError 

    mean_squared_error = residual_sum_of_squares/len(predictions)
    
    print( "Mean Squared Error (MSE) = ", mean_squared_error, "   (lower is better)" )
    
    return mean_squared_error   


def calculate_root_mean_squared_error ( residual_sum_of_squares, predictions) :
    if type( residual_sum_of_squares ) !=  numpy.float64 :
        raise TypeError
    
    if type( predictions ) !=  numpy.ndarray :
        raise TypeError 
    
    root_mean_squared_error  = numpy.sqrt(residual_sum_of_squares/ len(predictions) )
    print( "Root Mean Squared Error ( RMSE ) = ", root_mean_squared_error, "   (lower is better)" )
    return root_mean_squared_error


def get_null_prediction( dependent_variable_series ) :
    print( "type( dependent_variable_series ) = ", type( dependent_variable_series ))
    
    null_prediction = dependent_variable_series.mean()
    return null_prediction


def calculate_null_residuals( dependent_variable_series, null_prediction ) :
    print( "type( dependent_variable_series ) = ", type( dependent_variable_series ))
    print( "type( null_prediction ) = ", type( null_prediction ))
    
    null_residuals = dependent_variable_series - null_prediction
    return null_residuals


def calculate_null_residual_sum_of_squares( null_residuals ) :    
    print( "type( null_residuals ) = ", type( null_residuals ))
    
    null_residual_sum_of_squares = (null_residuals **2).sum()
    print( "Null residual sum of squares = ", null_residual_sum_of_squares )
    return calculate_null_residual_sum_of_squares


def calculate_and_plot_null_model_vs_your_model( dataframe, 
                                                linear_regression_model, 
                                                independent_variables_dataframe, 
                                                list_of_features, 
                                                dependent_variable_series ) :
    
    if type( dataframe ) != pandas.core.frame.DataFrame :
        raise TypeError
          
    if type( linear_regression_model ) != LinearRegression :
        raise TypeError 
        
    print( "type( independent_variables_dataframe ) = ", type( independent_variables_dataframe ))
    print( "type( list_of_features ) = ", type( list_of_features ))
    print( "type( dependent_variable_series ) =", type( dependent_variable_series ))
    
    model_score = linear_regression_model.score( independent_variables_dataframe, dependent_variable_series  )
    print( "Model Score = ", model_score, "   (higher is better)" )
    return model_score


def variance_inflation_factor(dataframe, feature1_name, feature2_name ) :
    if type( dataframe ) != pandas.core.frame.DataFrame :
        raise TypeError
          
    print( "type( feature1_name ) = ", type( feature1_name ))
    print( "type( feature2_name ) = ", type( feature2_name ))
    
    feature1_series = dataframe[ feature1_name ]
    feature2_series = dataframe[ feature2_name ]
    linear_regression_model = LinearRegression()
    linear_regression_model.fit( feature1_series, feature2_series )
    R2 = linear_regression_model.score( feature1_series , feature2_series  )
    variance_inflation_factor =  ( (1) / (1-R2) )
    return variance_inflation_factor


def calculate_regression_info( dataframe,  list_of_features, dependent_variable_name ) :
    if type( dataframe ) != pandas.core.frame.DataFrame :
        raise TypeError
          
    if type( list_of_features  ) !=  list :
        raise TypeError

    if type( dependent_variable_name ) !=  str :
        raise TypeError    

    
    independent_variables_dataframe = dataframe[ list_of_features ]
    dependent_variable_series = dataframe[ dependent_variable_name ]
    linear_regression_model = LinearRegression()
    linear_regression_model.fit( independent_variables_dataframe, dependent_variable_series )    
    
    return linear_regression_model
    
    
def calculate_regression_for_interaction_terms( dataframe,  
                                              list_of_features, 
                                              interaction_fieldname, 
                                              dependent_variable_series ) :
    
    if type( dataframe ) != pandas.core.frame.DataFrame :
        raise TypeError
          
    if type( list_of_features  ) !=  list :
        raise TypeError

    print( "type( interaction_fieldname ) = ", type( interaction_fieldname ))
    
    print( "type( dependent_variable_series ) = ", type( dependent_variable_series ))

    independent_variables_dataframe = dataframe[ list_of_features ]    
    for current_feature in list_of_features :
        try :
            if independent_variables_dataframe[interaction_fieldname] :
                print( "multiplying by term ", current_feature )
                independent_variables_dataframe[ interaction_fieldname ] = independent_variables_dataframe[ interaction_fieldname ] * dataframe[ current_feature ]
        except :        
            print( "creating dataframe with term ", current_feature )
            independent_variables_dataframe[interaction_fieldname] = dataframe[ current_feature ]
   
    return perform_linear_regression( dataframe, list_of_features, dependent_variable_name )


def perform_linear_regression( dataframe, list_of_features, dependent_variable_name, plots=[] ) :
    if type( dataframe ) != pandas.core.frame.DataFrame :
        raise TypeError
          
    if type( list_of_features  ) !=  list :
        raise TypeError

    if type( dependent_variable_name ) !=  str :
        raise TypeError
        
    independent_variables_dataframe = dataframe[ list_of_features ]
    dependent_variable_series = dataframe[ dependent_variable_name ]    
    linear_regression_model = calculate_regression_info( dataframe,  list_of_features, dependent_variable_name )
    predictions, residuals = get_predictions_and_residuals( dataframe, 
                                                                 linear_regression_model, 
                                                                 list_of_features, 
                                                                 dependent_variable_name )
    calculate_mean_absolute_error( residuals )
    residual_sum_of_squares = calculate_residual_sum_of_squares( residuals )
    mean_squared_error = calculate_mean_squared_error( residual_sum_of_squares, predictions )
    root_mean_squared_error = calculate_root_mean_squared_error( residual_sum_of_squares, predictions)
    null_prediction_array = get_null_prediction( independent_variables_dataframe )
    null_residuals = calculate_null_residuals( independent_variables_dataframe, null_prediction_array )
    null_residual_sum_of_squares = calculate_null_residual_sum_of_squares( null_residuals )
    
    R_squared = calculate_and_plot_null_model_vs_your_model( dataframe, 
                                                            linear_regression_model, 
                                                            independent_variables_dataframe, 
                                                            list_of_features, 
                                                            dependent_variable_series )
    
    if "coeficient" in plots :
        plot_coeficient_of_linear_regression_model( linear_regression_model )
        
    if "intercept" in plots :
        plot_intercept_of_regression_model( linear_regression_model )
     
    if "epsilon" in plots :
        plot_histogram_of_epsilon( dependent_variable_series, predictions )
    
    if "histogram" in plots :
        plot_histogram_of_dependent_variable( dataframe, independent_variables_dataframe )
        
    if "predicitons" in plots :
        plot_scatter_plot_of_predicitons_vs_residuals( predictions, residuals )
    
    if "regression" in plots :
        for series_name in list_of_features :
            seaborn.regplot(x=dependent_variable_name, y=series_name, data=dataframe );
            matplotlib.pyplot.show()        
       
    return linear_regression_model
    
    
def minimize_sse( dataframe, predictor_list, target_list ) :
    if type( dataframe ) != pandas.core.frame.DataFrame :
        raise TypeError
          
    print( "type( predictor_list ) = ", type( predictor_list ))
    print( "type( target_list ) = ", type( target_list ))    
    
    y = dataframe[ target_list ]
    X = dataframe[ predictor_list ]
    
    # Calculate the five "ingredients."
    y_bar = y.mean()
    x_bar = X.mean()
    std_y = y.std()
    std_x = X.std()
    r_xy = dataframe.corr().loc[ predictor_list, target_list ]

    # Calculate the betas.
    beta_1 = r_xy * std_y / std_x
    beta_0 = y_bar - beta_1 * x_bar

    yhat_linear = beta_0 + beta_1*X
    matplotlib.pyplot.scatter( X, y )
    matplotlib.pyplot.plot(X, yhat_linear, color='k');
    matplotlib.pyplot.show()
    
    return beta_0, beta_1


def hot_encode( dataframe, dictionary ) :
    if type( dataframe ) != pandas.core.frame.DataFrame :
        raise TypeError
          
    if type( dictionary ) != dict :
        raise TypeError
    
    for current_series_name in dictionary :
        try :
            dataframe = pandas.concat([dataframe, pandas.get_dummies(dataframe[current_series_name], prefix=dictionary[current_series_name] ) ], axis=1)
        except :
            print( "Something's wrong with the ", current_series_name, " series.")
            pass
    
    return dataframe

    
def get_train_and_test_metrics(dataframe, list_of_features, dependent_variable_name ) :
    if type( dataframe ) != pandas.core.frame.DataFrame :
        raise TypeError
          
    print( "type( list_of_features ) = ", type( dependent_variable_name ))
    
    features =[ c for c in dataframe._get_numeric_data().columns if not c in [dependent_variable_name]]
    X = dataframe[ features ] #all series except the independent variable
    y = dataframe[ dependent_variable_name ]

    X_train, X_test, y_train, y_test = train_test_split( X, y, random_state=42)

    return X, y, X_train, X_test, y_train, y_test


def compare_train_to_test( dataframe, model, list_of_features, dependent_variable_name  ) :
    if type( dataframe ) != pandas.core.frame.DataFrame :
        raise TypeError
          
    print( "type( model ) = ", type( model ))
    print( "type( list_of_features ) = ", type( list_of_features ))
    print( "type( dependent_variable_name ) = ", type( dependent_variable_name ))
    

    X, y, X_train, X_test, y_train, y_test = get_train_and_test_metrics(dataframe, list_of_features, dependent_variable_name )
    predict, residual = get_predictions_and_residuals( dataframe, model, X , y )
    return predict, residual
    
    
def get_features_matrix_and_target_vector( dataframe, 
                                          linear_regression_model, 
                                          independent_variable_list, 
                                          dependent_variable_name ) :
    
    if type( dataframe ) != pandas.core.frame.DataFrame :
        raise TypeError
          
    if type( linear_regression_model ) !=  LinearRegression :
        raise TypeError 

    if type( independent_variable_list ) !=  list :
        raise TypeError    

    if type( dependent_variable_name ) !=  str :
        raise TypeError    

    features =[ c for c in dataframe._get_numeric_data().columns if not c in [dependent_variable_name]]
    X = dataframe[ independent_variable_list ] #all series except the independent variable
    y = dataframe[ dependent_variable_name ]
    X_train, X_test, y_train, y_test = train_test_split( X, y, random_state=42)
    
    linear_regression_model.score( X_train, y_train)
    linear_regression_model.score( X_test, y_test)

    predict, residual = get_predictions_and_residuals( dataframe, linear_regression_model,
                                                      independent_variable_list,
                                                      dependent_variable_name )

    return predict, residual
    

columns_to_hot_encode = dict()
columns_to_hot_encode={ 'state' : 'state' }
analysis_dataframe=hot_encode( analysis_dataframe, columns_to_hot_encode )

In [7]:
analysis_dataframe.head()

,bias,center_lat,center_long,city,city_population,northeast_lat,northeast_long,southwest_lat,southwest_long,state,year
0,Anti-Male Homosexual,33.425510,-111.940005,TEMPE,156788.0,33.463984,-111.877226,33.319861,-111.978976,AZ,1996.0
1,Anti-Male Homosexual,34.483901,-114.322455,LAKE HAVASU CITY,34657.0,34.589585,-114.246145,34.430294,-114.375789,AZ,1996.0
2,Anti-Female Homosexual,32.222607,-110.974711,TUCSON,472385.0,32.320166,-110.708204,31.991654,-111.059406,AZ,1996.0
3,Anti-Male Homosexual,33.448377,-112.074037,PHOENIX,0.0,33.920570,-111.926046,33.290260,-112.324056,AZ,1996.0
4,Anti-Male Homosexual,37.871523,-122.273042,BERKELEY,101250.0,37.906689,-122.234196,37.835637,-122.368679,CA,1996.0


In [8]:
analysis_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43017 entries, 0 to 43016
Data columns (total 11 columns):
bias               43017 non-null object
center_lat         43017 non-null float64
center_long        43017 non-null float64
city               43017 non-null object
city_population    43017 non-null float64
northeast_lat      43017 non-null float64
northeast_long     43017 non-null float64
southwest_lat      43017 non-null float64
southwest_long     43017 non-null float64
state              43017 non-null object
year               43017 non-null float64
dtypes: float64(8), object(3)
memory usage: 3.6+ MB


In [34]:
crimes_by_location_population_by_year = dict()

iterations = 0
for iterations in range( 0 , analysis_dataframe.shape[0], 1) :

    population = analysis_dataframe.iloc[iterations, 4]
    if population==0 :
        continue

    year = analysis_dataframe.iloc[iterations, 10]
    city = analysis_dataframe.iloc[iterations, 3]
    state = analysis_dataframe.iloc[iterations, 9]
    northeast_lat = analysis_dataframe.iloc[iterations, 5]
    northeast_long = analysis_dataframe.iloc[iterations, 6]
    southwest_lat = analysis_dataframe.iloc[iterations, 7]
    southwest_long = analysis_dataframe.iloc[iterations, 8]
    
    if (iterations>0) & (iterations%100 == 0 ):
        print( iterations )
    iterations += 1  
       
 #   if crimes_by_location_population_by_year.keys() == 0 :
 #       crimes_by_location_population_by_year={ 
 #           year : {
 #               state : {
 #                   city : {
 #                       'population' : population, 
 #                       'crimes' : 1
 #                   },                
 #               }
 #           }
 #       }
 #   else :
    
    
    try :
        rec=crimes_by_location_population_by_year[year]
    except :
        crimes_by_location_population_by_year.update( { year : 
                                                   { state : 
                                                        { city : {
                                                                'population' : population,
                                                                'crimes' : 0,
                                                                'northeast_lat' : northeast_lat,
                                                                'northeast_long' : northeast_long,
                                                                'southwest_lat' : southwest_lat,
                                                                'southwest_long' : southwest_long
                                                            }
                                                        }
                                                   }
                                              } )

    try :
        rec=crimes_by_location_population_by_year[year][state]
    except :
        crimes_by_location_population_by_year[year].update( { state : 
                                                        { city : {
                                                                'population' : population,
                                                                'crimes' : 0,
                                                                'northeast_lat' : northeast_lat,
                                                                'northeast_long' : northeast_long,
                                                                'southwest_lat' : southwest_lat,
                                                                'southwest_long' : southwest_long
                                                            }
                                                        }
                                                   } )

    try :
        rec=crimes_by_location_population_by_year[year][state][city]
    except :
        crimes_by_location_population_by_year[year][state].update( { city : {
                                                                'population' : population,
                                                                'crimes' : 0,
                                                                'northeast_lat' : northeast_lat,
                                                                'northeast_long' : northeast_long,
                                                                'southwest_lat' : southwest_lat,
                                                                'southwest_long' : southwest_long
                                                            }
                                                        } )

    crime_count = crimes_by_location_population_by_year[year][state][city]['crimes']
    crimes_by_location_population_by_year[year][state][city]['crimes'] = crime_count + 1
        

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12600
12700
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17900
18000
18100
18200
18300
18400
18500
18600
18700
18800
18900
19000
19100
19300
19400
19500
19600
19700


In [35]:
print( crimes_by_location_population_by_year.keys() )

dict_keys([1996.0, 2011.0, 1999.0, 2004.0, 1997.0, 2002.0, 2015.0, 2003.0, 2012.0, 2006.0, 2016.0, 2005.0, 2014.0, 2008.0, 1998.0, 2001.0, 2000.0, 2013.0])


In [36]:
print( crimes_by_location_population_by_year[1996.0] )


{'AZ': {'TEMPE': {'population': 156788.0, 'crimes': 4, 'northeast_lat': 33.463984000000004, 'northeast_long': -111.8772261, 'southwest_lat': 33.319860800000001, 'southwest_long': -111.978976}, 'LAKE HAVASU CITY': {'population': 34657.0, 'crimes': 2, 'northeast_lat': 34.589585, 'northeast_long': -114.2461449, 'southwest_lat': 34.430293900000002, 'southwest_long': -114.37578899999998}, 'TUCSON': {'population': 472385.0, 'crimes': 2, 'northeast_lat': 32.320166100000002, 'northeast_long': -110.70820399999999, 'southwest_lat': 31.991653899999999, 'southwest_long': -111.059406}, 'BISBEE': {'population': 7251.0, 'crimes': 1, 'northeast_lat': 31.462218, 'northeast_long': -109.8674551, 'southwest_lat': 31.377623, 'southwest_long': -109.953309}, 'MESA': {'population': 340818.0, 'crimes': 2, 'northeast_lat': 33.513376000000001, 'northeast_long': -111.580636, 'southwest_lat': 33.277623900000002, 'southwest_long': -111.894881}, 'PHOENIX': {'population': 1139793.0, 'crimes': 24, 'northeast_lat': 33.

In [ ]:
crimes_by_location_population_by_year_dataframe = pandas.DataFrame()
for year in crimes_by_location_population_by_year.keys() :
    for state in crimes_by_location_population_by_year[year].keys() :
        for city in crimes_by_location_population_by_year[year][state].keys() :
            
            single_row_dictionary = { 
                "year" : year, 
                "state" : state, 
                "city" : city,
                "population" : crimes_by_location_population_by_year[year][state][city]["population"], 
                "crimes" : crimes_by_location_population_by_year[year][state][city]["crimes"], 
                "northeast_lat" : crimes_by_location_population_by_year[year][state][city]["northeast_lat"], 
                "northeast_long" : crimes_by_location_population_by_year[year][state][city]["northeast_long"], 
                "southwest_lat" : crimes_by_location_population_by_year[year][state][city]["southwest_lat"], 
                "southwest_long" : crimes_by_location_population_by_year[year][state][city]["southwest_long"]
             }
        
            crimes_by_location_population_by_year_dataframe = crimes_by_location_population_by_year_dataframe.append( single_row_dictionary,  ignore_index=True )

In [ ]:
calculate_regression_info( crimes_by_location_population_by_year,  [ 'crimes_by_location_population_by_year' ], 'crimes' )